<a href="https://colab.research.google.com/github/Samdersson/ACTIVIDAD-PRACTICA-RED-MULTICAPA_FUNCIONES-DE-ACTIVACI-N/blob/main/ACTIVIDAD_PRACTICA_RED_MULTICAPA_FUNCIONES_DE_ACTIVACI%C3%93N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ejercicio 1

In [ ]:
# 1. Importar las bibliotecas necesarias
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# 2. Cargar y preprocesar el dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# Normalizar las imágenes a valores entre 0 y 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# 3. Definir la arquitectura de la red neuronal convolucional (CNN)
model = models.Sequential()

# Primera capa convolucional
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))

# Segunda capa convolucional
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Capa de pooling
model.add(layers.MaxPooling2D((2, 2)))

# Capa de aplanamiento
model.add(layers.Flatten())

# Capa densa completamente conectada con 64 neuronas
model.add(layers.Dense(64, activation='relu'))

# Capa de salida con 10 neuronas (una para cada clase de CIFAR-10)
model.add(layers.Dense(10))

# 4. Compilar el modelo
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 5. Entrenar el modelo
history = model.fit(x_train, y_train, epochs=10,
                    validation_data=(x_test, y_test))

# 6. Evaluar el modelo
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

print(f'Precisión en el conjunto de prueba: {test_acc}')

# 7. Graficar el historial de precisión y pérdida
plt.plot(history.history['accuracy'], label='Precisión entrenamiento')
plt.plot(history.history['val_accuracy'], label = 'Precisión validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 186s 118ms/step - accuracy: 0.3925 - loss: 1.6618 - val_accuracy: 0.6086 - val_loss: 1.1121
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 196s 114ms/step - accuracy: 0.6344 - loss: 1.0450 - val_accuracy: 0.6460 - val_loss: 1.0139
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 206s 117ms/step - accuracy: 0.6902 - loss: 0.8918 - val_accuracy: 0.6664 - val_loss: 0.9580
Epoch 4/10
 764/1563 ━━━━━━━━━━━━━━━━━━━━ 1:26 109ms/step - accuracy: 0.7238 - loss: 0.7969

#ejercicio 2

In [ ]:
from IPython import get_ipython
from IPython.display import display
# Importar las bibliotecas necesarias
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Definir los hiperparámetros
input_size = 784
hidden_size = 128
output_size = 10
batch_size = 64
learning_rate = 0.001
num_epochs = 5

# Descargar y cargar el dataset MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Definir el modelo de red neuronal
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size): # Changed _init_ to __init__
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(-1, input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = NeuralNet(input_size, hidden_size, output_size)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Entrenar el modelo
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Época [{epoch+1}/{num_epochs}], Lote [{i+1}/{len(train_loader)}], Pérdida: {loss.item():.4f}')

# Evaluar el modelo
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Precisión en el conjunto de prueba: {100 * correct / total}%')